# Loading Arbitrary Curvilinear Data into yt

In this tutorial, we will use yt to perform some simple visualization tasks with curvilinear data. Make sure you have yt installed for this tutorial. If you don't, you can install it by following the instructions in the `installing-yt` notebook.

First, let's load the relevant libraries...

In [ ]:
import sys
sys.path.append('.local/lib/python3.5/site-packages')

In [ ]:
%matplotlib inline
import h5py
import numpy as np
import yt

We will use the same accretion disk GRMHD simulation as we did in a previous tutorial. Let's load it in just as we did before.

In [ ]:
grid = {}
with h5py.File('harmdisk2d/grid.h5','r') as f:
    for k,v in f.items():
        grid[k] = v.value
data = {}
with h5py.File('harmdisk2d/data.h5','r') as f:
    for k,v in f.items():
        data[k] = v.value

If your data is evenly spaced, you can load it via the `load_uniform_grid` method:

In [ ]:
help(yt.load_uniform_grid)

Unfortunately, our data is logarithmic in radius, so instead we will use the `load_hexahedral_mesh` method, which loads sime-structured grid data. The expectation for this method is that the data is of Cartesian product topology.

In [ ]:
help(yt.load_hexahedral_mesh)

We'll use the helper function `hexahedral_connectivity` to tell `yt` how the grid points are connected to each other:

In [ ]:
help(yt.hexahedral_connectivity)

First, we extract the 1d list of cell centers

In [ ]:
r = grid['Xbl'][:,0,0,1]*data['L_unit'][0]
theta = grid['Xbl'][0,:,0,2]
phi = grid['Xbl'][0,0,:,3]

The hexahedral mesh code actually wants cell faces, though. So we need to extract these. We can do so by finding `dx`.

In [ ]:
# r
dr = r[1]-r[0],r[-1]-r[-2]
rf = np.empty(len(r)+1)
rf[:-1] = r - dr[0]/2.
rf[-1] = rf[-2] + dr[1]
# theta
dth = theta[1]-theta[0],theta[-1]-theta[-2]
thf = np.empty(len(theta)+1)
thf[:-1] = theta - dth[0]/2.
thf[-1] = thf[-2] + dth[1]
# phi is special. It's only one zone and we know what the faces are
phif = np.array([0,2*np.pi])

Now we can generate the `coords` and `conn` data structures which tell us how grid vertices are connected to each other

In [ ]:
coords,conn = yt.hexahedral_connectivity(rf,thf,phif)

rf[-2]

And now we can load our data into yt

In [ ]:
arr_data = {'pressure':np.abs(data['PRESS'])*data['U_unit'] + 1e12, # to prevent underflow
            'entropy':data['ENT']}
bbox = np.array([[rf.min(),rf.max()],
                 [thf.min(),thf.max()],
                 [phif.min(),phif.max()]])
periodicity=[False,False,True]

ds = yt.load_hexahedral_mesh(arr_data,conn,coords,
                             #length_unit=data['L_unit'][0],
                             bbox=bbox,
                             periodicity=periodicity,
                             geometry='spherical')

By jumping through these few small hoops, we've enabled the ability to do some cool analysis tricks. For example, we can plot our data in spherical coordinates, which you may recall was an earlier exercise.

In [ ]:
slc = yt.SlicePlot(ds,'phi','pressure')
slc.save('../figures/harmdisk-xz-slice.pdf')
slc.show()

### Exercise:

Explore what data you can plot with yt using curvilinear coordinates. What works? What doesn't?